**Question-Answer using RNN in PyTorch.**

---



# UTILITY

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [2]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
# vocab
vocab = {'<UNK>':0}

def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)

In [4]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

# DATA DOWNLOAD & REVIEW

In [6]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


# DATA PREPROCESSING

In [7]:
df.apply(build_vocab, axis=1)
len(vocab)

324

# DATASET CLASS

In [8]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [9]:
# Create dataset object
dataset = QADataset(df, vocab)

In [10]:
# Create dataloader object
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# NEURAL NETWORK CLASS

In [12]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [13]:
learning_rate = 0.001
epochs = 20

model = SimpleRNN(len(vocab))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# TRAINING

In [14]:
# Training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Total Loss: {total_loss:4f}")

Epoch: 1, Total Loss: 526.420341
Epoch: 2, Total Loss: 461.560000
Epoch: 3, Total Loss: 383.157602
Epoch: 4, Total Loss: 317.812832
Epoch: 5, Total Loss: 264.681559
Epoch: 6, Total Loss: 216.500738
Epoch: 7, Total Loss: 172.675570
Epoch: 8, Total Loss: 134.348354
Epoch: 9, Total Loss: 103.899599
Epoch: 10, Total Loss: 79.824458
Epoch: 11, Total Loss: 61.876030
Epoch: 12, Total Loss: 48.361474
Epoch: 13, Total Loss: 38.742272
Epoch: 14, Total Loss: 31.440690
Epoch: 15, Total Loss: 26.658350
Epoch: 16, Total Loss: 22.570414
Epoch: 17, Total Loss: 19.257991
Epoch: 18, Total Loss: 16.678854
Epoch: 19, Total Loss: 14.594767
Epoch: 20, Total Loss: 12.725097


# MODEL EVALUATION

In [15]:
def predict(model, question, threshold=0.7):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")
  else:
    print(list(vocab.keys())[index])

In [18]:
predict(model, "Capital of France")

paris
